IMPORTS

In [124]:
import os
import requests
import json
import polars as pl
from key_YT import yt_key

SETTING PATH

In [125]:
# Change working directory to your specific folder
os.chdir('/content/drive/MyDrive/YouTube_Semantic_Search_ML/Data Engineering')

FUNCTIONS

In [126]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
        Function to extract YouTube video data from GET request response
    """

    video_record_list = []

    for raw_item in json.loads(response.text)['items']:

        # only execute for youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue

        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']
        video_record['description'] = raw_item['snippet']['description']

        video_record_list.append(video_record)

    return video_record_list

EXTARCT VIDEO IDs, DATETIME, TITLE

In [127]:
#get Channel ID
handle_name = "ShawhinTalebi"
url = f"https://www.googleapis.com/youtube/v3/channels?part=snippet&forHandle={handle_name}&key={yt_key}"
response = requests.get(url)
data = response.json()
channel_id = data['items'][0]['id']

# define url for API
url = 'https://www.googleapis.com/youtube/v3/search'

# initialize page token
page_token = None

# intialize list to store video data
video_record_list = []

In [128]:
%%time
# extract video data across multiple search result pages
while page_token != 0:

  # define parameters for API call
  params = {
      "key": yt_key,
      "channelId": channel_id,
      "part": ["snippet","id"],  # Change this to a comma-separated string
      "order": "date",
      "maxResults": 50,
      "pageToken": page_token
  }

  # make get request
  response = requests.get(url, params=params)

  # append video records to list
  video_record_list += getVideoRecords(response)

  try:
      # grab next page token
      page_token = json.loads(response.text)['nextPageToken']
  except:
      # if no next page token kill while loop
      page_token = 0

CPU times: user 31.9 ms, sys: 2.16 ms, total: 34.1 ms
Wall time: 971 ms


WRITE DATA TO FILE

In [129]:
# write data to file
pl.DataFrame(video_record_list).write_parquet('data/video-ids.parquet')
pl.DataFrame(video_record_list).write_csv('data/video-ids.csv')